In [1]:
import pandas as pd
import numpy as np
from functools import reduce
import matplotlib.pyplot as plt
import seaborn as sns
import time

%matplotlib inline
pd.set_option('display.max_columns', None)
pd.options.mode.chained_assignment = None  # To ignore SettingWithCopyWarning warning

# Import data

In [2]:
path = 'C:/Users/Akis/OneDrive/Masters/Web Economics/Data/WebEcon Team Drive/Datasets Original/'
train = pd.read_csv(path+"train.csv")
validation = pd.read_csv(path+"validation.csv")

In [3]:
print(train.shape, validation.shape)

(2430981, 25) (303925, 25)


# Import Predictions

In [4]:
path='C:/Users/Akis/OneDrive/Masters/Web Economics/Data/WebEcon Team Drive/predictions/'
# lr_preds = pd.read_csv("Prediction_csv/lr_pCTR.csv", index_col  = "Unnamed: 0")
# rf_preds = pd.read_csv("Prediction_csv/rf_pCTR.csv", index_col  = "Unnamed: 0")
####xgb_preds = pd.read_csv("Prediction_csv/xgb_pCTR.csv", index_col  = "Unnamed: 0")
# dwane_preds = pd.read_csv("Prediction_csv/click_predictions_narrow.csv")
# lasso_preds = pd.read_csv("Prediction_csv/lasso_basebid.csv", index_col  = "Unnamed: 0")
# ridge_preds = pd.read_csv("Prediction_csv/ridge_basebid.csv", index_col  = "Unnamed: 0")
# elasticnet_preds = pd.read_csv("Prediction_csv/elasticnet_basebid.csv", index_col  = "Unnamed: 0")
# rfg_preds = pd.read_csv("Prediction_csv/rfg_basebid.csv", index_col  = "Unnamed: 0")
####xgbr_preds = pd.read_csv("Prediction_csv/xgb_basebid.csv", index_col  = "Unnamed: 0")

xgb_preds = pd.read_csv(path+"xgb_pCTR.csv", index_col  = "Unnamed: 0")
xgbr_preds = pd.read_csv(path+"xgb_basebid.csv", index_col  = "Unnamed: 0")

In [5]:
xgbr_preds.head()

,bidid,predicted_payprice
0,bbcb813b6166538503d8b33a5602d7d72f6019dc,50.281219
1,5a07316c49477cb5d9b4d5aa39c27d6c3be7f92d,78.790169
2,f6ece71dae81d6b16bfb24ad6dd5611472d4c673,92.860153
3,b4d5c57c9b38ff5a12954fa01e11931b4e6bfbbb,45.761795
4,0899bf144249458ea9c89188473694bf44c7ca15,64.029907


In [6]:
print("Predicted Bid Price Min:",xgbr_preds['predicted_payprice'].min())
print("Predicted Bid Price Max:",xgbr_preds['predicted_payprice'].max())
print("Predicted Bid Price Mean:",xgbr_preds['predicted_payprice'].mean())
print("Predicted Bid Price Std:",xgbr_preds['predicted_payprice'].std())

Predicted Bid Price Min: -56.3769416809082
Predicted Bid Price Max: 346.9530944824219
Predicted Bid Price Mean: 78.97979187519267
Predicted Bid Price Std: 36.59964614741393


# Linear Strategy

In [33]:
def Strategy(sub_pCTR, sub_basebid, C ,BBimportance):
    
    importance = BBimportance * 10 + 10
    avgCTR = sub_pCTR["click_proba"].mean()
    
    mean = sub_basebid["predicted_payprice"].mean()
    sub_basebid["predicted_payprice"] = sub_basebid["predicted_payprice"]-mean
    sub_basebid["predicted_payprice"] = (importance * sub_basebid["predicted_payprice"]/sub_basebid["predicted_payprice"].max())
    sub_basebid["predicted_payprice"] = sub_basebid["predicted_payprice"] + mean
    #importance is a constant to be played around with
    
    avgBB = sub_basebid["predicted_payprice"].mean()
    
#     print("avBB/pBB Min:",(avgBB/sub_basebid["predicted_payprice"]).min())
#     print("avBB/pBB Max:",(avgBB/sub_basebid["predicted_payprice"]).max())
#     print("avBB/pBB Mean:",(avgBB/sub_basebid["predicted_payprice"]).mean())
#     print("avBB/pBB Std:",(avgBB/sub_basebid["predicted_payprice"]).std())
#     print()
    
#     print("pCTR/avCTR Min:",(sub_pCTR["click_proba"]/ avgCTR).min())
#     print("pCTR/avCTR Max:",(sub_pCTR["click_proba"]/ avgCTR).max())
#     print("pCTR/avCTR Mean:",(sub_pCTR["click_proba"]/ avgCTR).mean())
#     print("pCTR/avCTR Std:",(sub_pCTR["click_proba"]/ avgCTR).std())
    
    bidprice = (avgBB/sub_basebid["predicted_payprice"]) * (sub_pCTR["click_proba"]/ avgCTR) * C
    return bidprice

In [34]:
Strategy(xgb_preds, xgbr_preds, 100, 0) # 1 to 15 is a good range

0           15.417813
1           19.871407
2            1.921149
3           10.822750
4           11.933050
5          222.320845
6           53.629591
7          103.140723
8           13.357650
9            5.340924
10         139.699448
11          27.820761
12           6.971949
13         136.470960
14         176.403206
15           9.898831
16          11.673160
17          19.447115
18         549.816393
19          58.989825
20          61.240884
21          60.241915
22          37.893543
23          23.736468
24         156.570440
25         337.714120
26           4.141494
27         120.420126
28          25.271011
29          93.967053
             ...     
303895    1312.089074
303896       5.337643
303897     595.556926
303898     150.327952
303899     152.553343
303900      55.362043
303901       3.268741
303902      72.840576
303903       8.873061
303904      56.412230
303905      34.589144
303906     194.167417
303907      19.705905
303908      17.735084
303909    

In [9]:
base_bid = train.loc[ train["click"] == 1, "payprice"].mean()
base_bid

105.46402677077523

In [47]:
base_bid = 89
basebids = np.linspace(0.90*base_bid, 1.06*base_bid, 8)
basebids = np.linspace(base_bid-2, base_bid+6, 9)
basebids

array([87., 88., 89., 90., 91., 92., 93., 94., 95.])

# Evaluate Results

In [48]:
budget = 6250*1000

def ValidationDataFrame(submission_pCTR, submission_basebid, C, BBimportance):
    
    validation_check = validation[["bidid", "click", "bidprice", "payprice"]]
    validation_check["click_proba"] = submission_pCTR["click_proba"]
    validation_check["bidprice_predicted"] = Strategy(submission_pCTR, submission_basebid, C , BBimportance)
    
    return validation_check


def ValidateStrategy(df):
    
    impressions = 0
    clicks = 0
    cost = 0
    auctions_participated = 0
    balance = budget
    
    for row in df.iterrows():
        
        if cost < budget:
            
            auctions_participated+=1
            
            if (row[1]["bidprice_predicted"] >= row[1]["payprice"]):
                
                if (balance > row[1]["bidprice_predicted"]):
                    
                    impressions+=1
                    clicks+=row[1]["click"]
                    cost+=row[1]["payprice"]
                    balance-=row[1]["payprice"]
                else:
                    pass
    
        else:
            break
    
    # Metrics
    ctr = clicks*100/impressions
    cpm = cost/impressions
    cpc = cost/clicks/1000
    
    print("Strategy statistics:")
    print("Auctions participated: {} | Impressions: {} | Clicks: {} | Cost: {} | CTR: {} | CPM: {} | CPC: {}".format(auctions_participated, \
                                                                                                                     impressions, clicks, \
                                                                                                                     cost,\
                                                                                                                     np.round(ctr,5), \
                                                                                                                     np.round(cpm,5), \
                                                                                                                     np.round(cpc,2)))
    print("\n")
            
    return impressions, clicks, cost, auctions_participated, ctr, cpm, cpc

In [49]:
pCTR_model_name = "XGBoost"
results = pd.DataFrame(columns = ["pCTR_model", "Coefficient", "Impressions", "Clicks", "Cost", "CTR", "CPM", "CPC"])
i=0

start = time.time()

for C in basebids:
    for BBIMPORTANCE in range(0,2,1):
    
        validation_check = ValidationDataFrame(xgb_preds, xgbr_preds, C, BBIMPORTANCE)
        print("\033[1m pCTR model: {} \033[0m, \033[1m Constant: {} \033[0m".format(pCTR_model_name, C))
        print("pBB Importance:",BBIMPORTANCE)
        impressions, clicks, cost, auctions_participated, ctr, cpm, cpc = ValidateStrategy(validation_check)
        results.loc[i] = [pCTR_model_name, C, impressions, clicks, cost, ctr, cpm, cpc]
        i+=1
    
end = time.time()
print("Total time: {} mins".format((end-start)/60))

 pCTR model: XGBoost ,  Constant: 87.0 
pBB Importance: 0
Strategy statistics:
Auctions participated: 303925 | Impressions: 112795 | Clicks: 160 | Cost: 5858352 | CTR: 0.14185 | CPM: 51.93805 | CPC: 36.61


 pCTR model: XGBoost ,  Constant: 87.0 
pBB Importance: 1
Strategy statistics:
Auctions participated: 303925 | Impressions: 112795 | Clicks: 160 | Cost: 5858352 | CTR: 0.14185 | CPM: 51.93805 | CPC: 36.61


 pCTR model: XGBoost ,  Constant: 88.0 
pBB Importance: 0
Strategy statistics:
Auctions participated: 303925 | Impressions: 113595 | Clicks: 160 | Cost: 5917151 | CTR: 0.14085 | CPM: 52.08989 | CPC: 36.98


 pCTR model: XGBoost ,  Constant: 88.0 
pBB Importance: 1
Strategy statistics:
Auctions participated: 303925 | Impressions: 113595 | Clicks: 160 | Cost: 5917151 | CTR: 0.14085 | CPM: 52.08989 | CPC: 36.98


 pCTR model: XGBoost ,  Constant: 89.0 
pBB Importance: 0
Strategy statistics:
Auctions participated: 303925 | Impressions: 114455 | Clicks: 160 | Cost: 5977235 | CTR: 0.13

In [ ]:
results.to_csv("ValidationSet_Evaluation_Results_Individual_xgb_xcgr.csv")

In [ ]:
results.head()

In [ ]:
results.sort_values(by = ["Clicks",'CTR'], ascending = False)

In [ ]:
plt.figure(figsize = (10,8))

plt.subplot(4,1,1)
plt.plot(basebids, results["Clicks"])
plt.ylabel("Clicks")

plt.subplot(4,1,2)
plt.plot(basebids, results["Impressions"])
plt.ylabel("Impressions")

plt.subplot(4,1,3)
plt.plot(basebids, results["CTR"])
plt.ylabel("CTR")

plt.subplot(4,1,4)
plt.plot(basebids, results["Cost"])
plt.ylabel("Cost")
plt.xlabel("Constant")

plt.tight_layout()